In [1]:
import numpy as np
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train, y_train = load_arrow_head(split="test", return_X_y=True)
X_test, y_test = load_arrow_head(split="train", return_X_y=True)
print(X_train.shape, X_test.shape)

(175, 1) (36, 1)


In [3]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
    is_nested_dataframe,
)

x_np = from_nested_to_2d_array(X_train)
x = x_np.iloc[:10, :10] # Using first 5 steps for faster training
X_train = from_2d_array_to_nested(x)


In [4]:
# ECG dataset
# from sktime.datasets import load_from_arff_to_dataframe
# from IPython.display import display

# X_train, y_train = load_from_arff_to_dataframe('./ECG5000/ECG5000_TRAIN.arff')
# X_test, y_test = load_from_arff_to_dataframe('./ECG5000/ECG5000_TEST.arff')


In [5]:
rocket = RocketClassifier()
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
accuracy_score(y_test, y_pred)

0.3333333333333333

In [6]:

X_pyreal = np.empty((X_train.shape[0], X_train.iloc[0][0].shape[0]))
print(X_pyreal.shape)
for i in range(X_train.shape[0]):
    for j in range(X_train.iloc[0][0].shape[0]):
        X_pyreal[i,j] = X_train.iloc[i][0][j]

X_pyreal = pd.DataFrame(X_pyreal)

(175, 10)


In [7]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested
)
class wrapped_model(object):
    def __init__(self, model):
        self.model = model

    def __call__(self, X):
        self.predict(X)

    def transform(self, data):
        df = from_2d_array_to_nested(data)
        return df

    def predict(self, X):
        X = self.transform(X)
        # without astype, the output is in <u1
        return self.model.predict(X).astype('int')

model = wrapped_model(rocket)
print(model.predict(X_pyreal))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:

from pyreal.explainers import TimeSeriesImportance

# change window_size to aggregate different timesteps
explainer = TimeSeriesImportance(model, X_pyreal, window_size=3)
explainer.fit()
explanation, X_interpret = explainer.produce(X_pyreal.iloc[:1, :])

print(explanation)


Using 175 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1 [00:00<?, ?it/s]